# Tensorflow 1버전에 맞춘 Transformer 모듈


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install --upgrade keras
!pip install fasttext==0.8.3
import fasttext

from tensorflow import keras
import os
import sys
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text
from keras import metrics
from keras import backend
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
import keras.layers as layers
from keras import optimizers
import re
import string

     |████████████████████████████████| 378kB 4.8MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5
     |████████████████████████████████| 81kB 3.8MB/s 
  Created wheel for fasttext: filename=fasttext-0.8.3-cp36-cp36m-linux_x86_64.whl size=1092678 sha256=fb50dc513b9fabbafc5944a411d5d3e5fb8a66534dc390aa4642eebee455b8a6
  Stored in directory: /root/.cache/pip/wheels/73/8e/5d/ecb50b90adaab5868ae1d8df180f31e55e85c2f055aaf2fb35
Successfully built fasttext


Using TensorFlow backend.


In [4]:
import tensorflow as tf
from tensorflow import keras
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.8
tf.enable_eager_execution(config=config)

print(keras.__version__)
print(tf.__version__)

# keras와 tf 버전이 중요합니다
# Requirements.txt
#keras==2.1.2
#numpy==1.13.1
#fasttext==0.8.3
#tensorflow-gpu==1.4.1

# 기본 코랩 버전으로도 비벼진다는 점을 발견했습니다
#2.2.4-tf
#1.15.0

2.2.4-tf
1.15.0


In [0]:
pip list

Package                  Version    
------------------------ -----------
absl-py                  0.9.0      
alabaster                0.7.12     
albumentations           0.1.12     
altair                   4.0.0      
asgiref                  3.2.3      
astor                    0.8.1      
astropy                  4.0        
atari-py                 0.2.6      
atomicwrites             1.3.0      
attrs                    19.3.0     
audioread                2.1.8      
autograd                 1.3        
Babel                    2.8.0      
backcall                 0.1.0      
backports.tempfile       1.0        
backports.weakref        1.0.post1  
beautifulsoup4           4.6.3      
bleach                   3.1.0      
blis                     0.2.4      
bokeh                    1.4.0      
boto                     2.49.0     
boto3                    1.10.47    
botocore                 1.13.47    
Bottleneck               1.3.1      
branca                   0.3.1      
b

## 모듈



In [0]:
from tensorflow import keras
import os
import json
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text

# Preprocess - 토크나이저와 전처리를 맡는 클래스! 순서대로 진행 추천
# 먼저 3개 인자 넣음 (Q와 A는 pandas series type일 때 작동확인)
# 1. buildTokenizer : 토크나이저 리턴(save 가능)
# 2. tokenize_and_filter : 토크나이징 된 input과 output 리턴

class Preprocess:
    def __init__(self):
        super().__init__()
    def buildTokenizer(self, questions, answers):
        corpus = questions + answers
        tk = Tokenizer(filters='!"#$%&()*+,-.s:;<=>?@[\\]^_`{|}~\t\n', lower=False) # / 만 살렸다, 소문자변환 안한다.
        tk.fit_on_texts(corpus)
        VOCAB_SIZE = len(tk.word_index) + 1
        self.START_TOKEN, self.END_TOKEN = [VOCAB_SIZE], [VOCAB_SIZE + 1]
        self.VOCAB_SIZE = VOCAB_SIZE + 2   
        return tk

    def saveTokenizer(self, directory, tokenizer):
        tk_json = tokenizer.to_json()
        with open(directory, 'w', encoding='UTF-8-sig') as f:
            f.write(json.dumps(vars(tokenizer), ensure_ascii = False))
    def loadTokenzier(self, directory):
        with open(directory, encoding='UTF-8-sig') as fh:
            data = json.load(fh)
        tk = Tokenizer()
        key = list(data.keys())
        for i in key:
            setattr(tk, i, data[i])
        VOCAB_SIZE = len(tk.word_index) + 1 
        self.START_TOKEN, self.END_TOKEN = [VOCAB_SIZE], [VOCAB_SIZE + 1]
        self.VOCAB_SIZE = VOCAB_SIZE + 2  
        return tk

    # 리스트 형태를 토큰 모델에 넣어주려고 text로 만듦
    def list2text(self, x):
        return [e for s in x for e in s]

    # Tokenize, filter and pad sentences
    def tokenize_and_filter(self, questions, answers, tokenizer, MAX_LENGTH=30):
        tokenized_inputs, tokenized_outputs = [], []

        for (sentence1, sentence2) in zip(questions, answers):
            # tokenize sentence
            sentence1 = [self.START_TOKEN] + tokenizer.texts_to_sequences(sentence1) + [self.END_TOKEN]
            sentence2 = [self.START_TOKEN] + tokenizer.texts_to_sequences(sentence2) + [self.END_TOKEN]
            # check tokenized sentence max length
            if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
                tokenized_inputs.append(self.list2text(sentence1))
                tokenized_outputs.append(self.list2text(sentence2))

        # pad tokenized sentences
        tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
                            tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
        tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
                            tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
        return tokenized_inputs, tokenized_outputs
    
    def buildDataset(self, inputs, outputs, BATCH_SIZE, BUFFER_SIZE=20000):
        dataset = tf.data.Dataset.from_tensor_slices((
            {
                'inputs': inputs,
                'dec_inputs': outputs[:, :-1]
            },
            {
                'outputs': outputs[:, 1:]
            },
        ))
        dataset = dataset.cache()
        dataset = dataset.repeat()
        dataset = dataset.shuffle(BUFFER_SIZE)
        dataset = dataset.batch(BATCH_SIZE)
        dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
        return dataset

############################
#### MultiHeadAttention ####
############################
def scaled_dot_product_attention(query, key, value, mask):
    """Calculate the attention weights. """
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # scale matmul_qk
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth) # scaling 

    # add the mask to zero out padding tokens
    if mask is not None:
        logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k)
    attention_weights = tf.nn.softmax(logits, axis=-1) 

    output = tf.matmul(attention_weights, value)

    return output

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, name="multi_head_attention", **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = self.d_model // self.num_heads
        assert d_model % self.num_heads == 0
        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)
        self.dense = tf.keras.layers.Dense(units=d_model)

    def get_config(self):
        config = super(MultiHeadAttention, self).get_config()
        config.update({
                        "num_heads":self.num_heads, 
                        "d_model":self.d_model,
                       })
        return config

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[ 
                                    'value'], inputs['mask']
        batch_size = tf.shape(query)[0]
        # linear layers
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)
        # split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)
        # scaled dot-product attention
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        # concatenation of heads
        concat_attention = tf.reshape(scaled_attention,
                                        (batch_size, -1, self.d_model))
        outputs = self.dense(concat_attention)
        return outputs

#################
#### Masking ####
#################
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

######################
#### Pos Encoding ####
######################
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model, name='PositionalEncoding', **kwargs):
        super(PositionalEncoding, self).__init__(**kwargs)
        self.d_model = d_model
        self.position = position
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(tf.constant(10000,dtype='float32'), (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
                            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
                            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
                            d_model=d_model)
        # apply sin to even index in the array
        sines = tf.math.sin(angle_rads[:, 0::2])
        # apply cos to odd index in the array
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def get_config(self):
        config = super(PositionalEncoding, self).get_config().copy()
        config.update({
                        'd_model': self.d_model,
                        'position': self.position
                        })
        return config

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

###########################
#### 인코더 내부 Layer ####
###########################
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    attention = MultiHeadAttention(
        d_model, num_heads, name="attention")({
            'query': inputs,
            'key': inputs,
            'value': inputs,
            'mask': padding_mask
        })
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attention)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
        inputs=[inputs, padding_mask], outputs=outputs, name=name)
  
##############################################
#### 임베딩 + Pos Encoding + 인코더 Layer ####
##############################################
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, padding_mask], outputs=outputs, name=name)

###########################
#### 디코더 내부 Layer ####
###########################
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(d_model, num_heads, name="attention_1")(inputs={
                    'query': inputs,
                    'key': inputs,
                    'value': inputs,
                    'mask': look_ahead_mask
                })
    attention1 = tf.keras.layers.LayerNormalization(
    epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(d_model, num_heads, name="attention_2")(inputs={
                    'query': attention1,
                    'key': enc_outputs,
                    'value': enc_outputs,
                    'mask': padding_mask
                })
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
    epsilon=1e-6)(attention2 + attention1)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
    epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
                    inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
                    outputs=outputs,
                    name=name)

###################################################
#### 임베딩 + Pos Encoding + 디코더 내부 Layer ####
###################################################
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
                    units=units,
                    d_model=d_model,
                    num_heads=num_heads,
                    dropout=dropout,
                    name='decoder_layer_{}'.format(i),
                )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
                        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
                        outputs=outputs,
                        name=name)

#######################################
#### 인코더 + 디코더 = Transformer ####
#######################################
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    enc_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask, output_shape=(1, 1, None),
        name='enc_padding_mask')(inputs)
    # mask the future tokens for decoder inputs at the 1st attention block
    look_ahead_mask = tf.keras.layers.Lambda(
        create_look_ahead_mask,
        output_shape=(1, None, None),
        name='look_ahead_mask')(dec_inputs)
    # mask the encoder outputs for the 2nd attention block
    dec_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask, output_shape=(1, 1, None),
        name='dec_padding_mask')(inputs)

    enc_outputs = encoder(
                    vocab_size=vocab_size,
                    num_layers=num_layers,
                    units=units,
                    d_model=d_model,
                    num_heads=num_heads,
                    dropout=dropout,
                )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
                    vocab_size=vocab_size,
                    num_layers=num_layers,
                    units=units,
                    d_model=d_model,
                    num_heads=num_heads,
                    dropout=dropout,
                )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)
    
#######################################
######### Loss & LearningRate #########
#######################################
def customLoss(MAX_LENGTH, name='customLoss'):
    def loss_function(y_true, y_pred, name='loss_function'):
        y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

        loss = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction='none')(y_true, y_pred)

        mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
        loss = tf.multiply(loss, mask)
        return tf.reduce_mean(loss)
    return loss_function
    
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000, name='CustomSchedule', **kwargs):
        super(CustomSchedule, self).__init__(**kwargs)

        self.d_m = d_model
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        config = {
                  'd_model': self.d_m,
                  'warmup_steps': self.warmup_steps,
                 }
        return config 

def custom_accuracy(MAX_LENGTH, name='custom_accuracy'):
    def accuracy(y_true, y_pred):
        y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
        return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)
    return accuracy

#### 예측 ####
def transform(sentence, analyzer, anl_type):
    if anl_type == 'Konlpy_morphs':
        return [[i] for i in  analyzer.morphs(sentence)]
    elif anl_type == 'Konlpy_pos':
        return [[i] for i in  analyzer.pos(sentence, join=True)]
    elif anl_type == 'Khaiii':
        sentence = [str(i).split('\t')[1] for i in  analyzer.analyze(sentence)]
        return [e for s in sentence for e in s.split(' + ')]
    else:
        print('use Konlpy or Khaiii')

def evaluate(model, tokenizer, sentence, analyzer, anl_type, MAX_LENGTH=30):
    VOCAB_SIZE = len(tokenizer.word_index) + 1
    START_TOKEN, END_TOKEN = [VOCAB_SIZE], [VOCAB_SIZE + 1]
    VOCAB_SIZE += 2

    sentence = [START_TOKEN] + tokenizer.texts_to_sequences(transform(sentence, analyzer, anl_type)) + [END_TOKEN]
    sentence = tf.expand_dims([e for s in sentence for e in s],axis=0)
    output = tf.expand_dims(START_TOKEN, 0)
    for i in range(MAX_LENGTH):
        predictions = model(inputs=[sentence, output], training=False)

    # select the last word from the seq_len dimension
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
    # return the result if the predicted_id is equal to the end token
        if tf.equal(predicted_id, END_TOKEN[0]) is None:
            break
        output = tf.concat([output, predicted_id], axis=-1)

    output = tf.squeeze(output, axis=0)
    return tokenizer.sequences_to_texts(
                        [[i] for i in output.numpy()])

def pos2text(token_list):
    pure_input = [i[:i.index('/')] for i in token_list]
    # 띄어쓰기 교정
    # Josa : 조사, PreEomi : 선어말어미, Eomi : 어미, Suffix : 접미사
    pos_list = ['Josa','PreEomi','Eomi','Suffix']
    for i in range(len(token_list)):
        p = token_list[i]
        if p[p.index('/')+1:] in pos_list:
            # '/토큰'으로 만들어서 나중에 앞 단어와 붙일 거임
            pure_input[i] = '/'+pure_input[i]
    pure_text = ' '.join(pure_input)
    pure_text = pure_text.replace(' /','')

    # Rule 추가
    pure_text = pure_text.replace('중 증','중증')
    pure_text = pure_text.replace(' 은','은')
    pure_text = pure_text.replace(' 로','로')
    pure_text = pure_text.replace(' 와','와')


    return pure_text

def token2text(token_list):
    return ' '.join(token_list)
def text2token(x, analyzer):
    return analyzer.pos(x, join=True)

## Train

In [0]:
# 사람마다 디렉토리 다름 주의
tk_dir = '/content/drive/My Drive/ToBigs/탄시리(Tanhsiri)/Phase2 : Modeling & Algorithm & Advanced Modeling/Transformer_준걸_윤종/tokenizer_data_v1.json'
corpus = '/content/drive/My Drive/ToBigs/탄시리(Tanhsiri)/Phase1 : Data Collection & Preprocessing /Data/통합데이터/Konlpy_data.pickle'
df = pd.read_pickle(corpus)

# Khaiii 썼을 떄 쓰는 거
# def cut(x):
#     return [i[:i.index('/')+3] for i in x]
# questions = df['pre_Question'].apply(cut)
# answers = df['pre_Answer'].apply(cut)
# questions = df['Q_kh']
# answers = df['A_kh']

# Konlpy 썼을 떄 쓰는 거
questions = df['Q_noPos']
answers = df['A_noPos']

MAX_LENGTH = 30
preprocess = Preprocess()

# 옵션1) 새로운 토크나이저 만들기
# tokenizer = preprocess.buildTokenizer(questions, answers)
# questions_seq, answers_seq = preprocess.tokenize_and_filter(questions, answers, tokenizer, MAX_LENGTH)
# 옵션2) 토크나이저 저장
# preprocess.saveTokenizer(tk_dir, tokenizer)
# 옵션3) 기존 토크나이저 로드
tokenizer = preprocess.loadTokenzier(tk_dir)
questions_seq, answers_seq = preprocess.tokenize_and_filter(questions, answers, tokenizer, MAX_LENGTH)

print('Vocab size: {}'.format(preprocess.VOCAB_SIZE))
print('Number of samples: {}'.format(len(questions_seq)))

Vocab size: 3663
Number of samples: 1043


In [0]:
# dataset
BATCH_SIZE = 64
dataset = preprocess.buildDataset(questions_seq, answers_seq, BATCH_SIZE)
dataset

<BatchDataset shapes: ({inputs: (?, 30), dec_inputs: (?, 29)}, {outputs: (?, 29)}), types: ({inputs: tf.int32, dec_inputs: tf.int32}, {outputs: tf.int32})>

In [0]:
# tf.keras.backend.clear_session()

# parameters
NUM_LAYERS = 2
D_MODEL = 512
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.2
VOCAB_SIZE = preprocess.VOCAB_SIZE

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

learning_rate = 1e-4

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

model.compile(optimizer=optimizer, loss=customLoss(MAX_LENGTH), metrics=[custom_accuracy(MAX_LENGTH)])

# 옵션) 모델 plot
# tf.keras.utils.plot_model(model, to_file='transformer.png', show_shapes=True)

TypeError: ignored

In [0]:
steps = len(questions_seq) // BATCH_SIZE
EPOCHS = 100
print(f'MAX_LENGTH : {MAX_LENGTH}, EPOCHS : {EPOCHS}')

model.fit(dataset, epochs=EPOCHS, steps_per_epoch=steps)

# 옵션) verbose = 0 모드일 떄 최종 train acc 찍어보는 거
# test_scores = model.evaluate(dataset, verbose=2)
# print('Test loss:', test_scores[0])
# print('Test accuracy:', test_scores[1])

MAX_LENGTH : 30, EPOCHS : 100
Train on 16 steps
Epoch 1/100
16/16 [==============================] - 5s 292ms/step - loss: 4.4979 - accuracy: 6.7349e-05
Epoch 2/100
16/16 [==============================] - 1s 59ms/step - loss: 4.3856 - accuracy: 0.0049
Epoch 3/100
16/16 [==============================] - 1s 59ms/step - loss: 4.2734 - accuracy: 0.0315
Epoch 4/100
16/16 [==============================] - 1s 58ms/step - loss: 4.1639 - accuracy: 0.0346
Epoch 5/100
16/16 [==============================] - 1s 59ms/step - loss: 3.9699 - accuracy: 0.0345
Epoch 6/100
16/16 [==============================] - 1s 61ms/step - loss: 4.0237 - accuracy: 0.0348
Epoch 7/100
16/16 [==============================] - 1s 59ms/step - loss: 4.0084 - accuracy: 0.0365
Epoch 8/100
16/16 [==============================] - 1s 60ms/step - loss: 3.8663 - accuracy: 0.0400
Epoch 9/100
16/16 [==============================] - 1s 58ms/step - loss: 3.7261 - accuracy: 0.0404
Epoch 10/100
16/16 [===========================

In [0]:
# 모델 저장
path = '/content/drive/My Drive/ToBigs/탄시리(Tanhsiri)/Phase2 : Modeling & Algorithm & Advanced Modeling/Transformer_준걸_윤종/tf_model_v1.h5'
model.save(path)
del model

## Eval

In [5]:
# Install Konlpy
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [81.6 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://ppa.launchpad.net/mar

In [0]:


# 모델 로드
path_1 = '/content/drive/My Drive/ToBigs/탄시리(Tanhsiri)/Phase2 : Modeling & Algorithm & Advanced Modeling/Transformer_준걸_윤종/tf_model_noPos3.h5'
MAX_LENGTH = 30
custom_objects = {'PositionalEncoding':PositionalEncoding,
                'MultiHeadAttention':MultiHeadAttention,
                'CustomSchedule':CustomSchedule,
                'loss_function':customLoss(MAX_LENGTH),
                'accuracy':custom_accuracy(MAX_LENGTH), 
                'create_padding_mask':create_padding_mask,
                'backend':backend, 
                'tf':tf}
model_1 = keras.models.load_model(path_1, custom_objects=custom_objects)

# 토크나이저 로드
tk_dir_1 = '/content/drive/My Drive/ToBigs/탄시리(Tanhsiri)/Phase2 : Modeling & Algorithm & Advanced Modeling/Transformer_준걸_윤종/tokenizer_data_Konlpy_noPos.json'

preprocess1 = Preprocess()
tokenizer_1 = preprocess1.loadTokenzier(tk_dir_1)

# 토크나이저 로드하면 모든 key,value가 string으로 들어감 나중에 토큰을 텍스트로 
# 복원할 때 정상적으로 구동하기 위해서 index_word는 key를 int로 바꿔줌
tokenizer_1.index_word = {int(k):v for k,v in tokenizer_1.index_word.items()}

In [0]:
from konlpy.tag import Okt
t = Okt()

# 띄어쓰기

class Space:
    def __init__(self):
        # 띄어쓰기
        self.adam_half = optimizers.Adam(lr=0.0005)
        self.dic_kor = self.load_bin()
        self.model_kor = self.load_space_model()
        self.threshold_kor = 0.5
        self.overlap = 30

    def load_bin(self):
        return fasttext.load_model('/content/drive/My Drive/model_kor.bin')
    def load_space_model(self):
        return keras.models.load_model('/content/drive/My Drive/model_kor.hdf5')

    def pred_correction_rnn(self, sent,model,dic,maxlen,wdim):
        conv = np.zeros((1,maxlen,wdim,1))
        rnn = np.zeros((1,maxlen,wdim))
        charcount = -1
        for j in range(len(sent)):
          print('j')
          if j<maxlen and sent[j]!=' ':
            charcount=charcount+1
            conv[0][charcount,:,0]=dic[sent[j]]
            rnn[0][charcount,:]=dic[sent[j]]
        z = model.predict([conv,rnn])[0]
        sent_raw = ''
        count_char=-1
        lastpoint=-1
        lastchar=-1
        for j in range(len(sent)):
          if sent[j]!=' ':
            count_char=count_char+1
            sent_raw = sent_raw+sent[j]
            if z[count_char]>self.threshold_kor:
              sent_raw = sent_raw+' '
              if j<self.overlap:
                lastpoint=len(sent_raw)
                lastchar=j
        return sent_raw, lastpoint, lastchar

    def kor_spacing(self, s):
        if len(s)<self.overlap:
          temp,lp,lc = self.pred_correction_rnn(s,self.model_kor,self.dic_kor,100,100)
          z = temp+"\n"
        else:
          z=''
          start=0
          while start<len(s):
            if start+self.overlap<len(s):
              print('sent', s[start:start+2*self.overlap])
              temp,lp,lc =self.pred_correction_rnn(s[start:start+2*self.overlap],self.model_kor,self.dic_kor,100,100)
              temp=temp[:lp]
            else:
              temp,lp,lc =self.pred_correction_rnn(s[start:],self.model_kor,self.dic_kor,100,100)
              lc = self.overlap
            z = z+temp
            start=start+lc+1
          z = z+"\n"
        return z



In [22]:
space = Space()

AttributeError: ignored

In [17]:
# Konlpy

# while True:
#     sentence = input()
#     if sentence == 'quit':
#         break
sentence = '장애인도신체검사받나요'
prediction_1 = evaluate(model_1, tokenizer_1, sentence, t, anl_type='Konlpy_morphs', MAX_LENGTH=30)
new_text_2 = space.kor_spacing(token2text(prediction_1))
# print(f'Q : {sentence}')
# print(f'Konlpy : {token2text(prediction_1)}')
print(f'After Spaceing : {new_text_2}')


sent  장애인 은 병역 이 면제 되나 장애인 등록 자 중 병역 판정 검사 대상자 의 경우 병역 판정 검사 를 받아
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
sent  중 병역 판정 검사 대상자 의 경우 병역 판정 검사 를 받아야 합니다  를 받아야 합니다    
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
j
After Spaceing : 장애인은 병역이 면제되나 장애인 등록자 중병역 판정검사 대상자의 경우병역판정 검사를 받아야 합니다를 받아야 합니다



In [20]:
list(space.dic_kor.words)
# space.model_kor

['사랑하든',
 '했나',
 '했을까봐',
 '축하금',
 '봉이씨랑',
 '연인',
 '라고도',
 '가져오라고',
 '대답부터',
 '아무사이',
 '그만이니까',
 '학교폭력',
 '됐',
 '만노군',
 '군포를',
 '그랬다니까',
 '얻게',
 '생각함',
 '더러운',
 '괘씸죄로',
 '선생께서',
 '있었다면서',
 '두달에',
 '일천한',
 '되느니',
 '공약으로',
 '못합니까',
 '만나면서',
 '좋아하지두',
 '옮긴',
 '꼼짝두',
 '따라갈게요',
 '공맹의',
 '참',
 '코피도',
 '나뭇잎',
 '휴가예요',
 '가서라도',
 '가둔',
 '사람하고',
 '애쓰셨어요',
 '전화한거야',
 '그런뜻이',
 '아침입니다',
 '부러우세요',
 '시작했잖아',
 '지경사',
 '모두요',
 '명월이',
 '가볍고',
 '노려보면',
 '속이다니',
 '팔만',
 '달랍니다',
 '설거지까지',
 '만만찮게',
 '사랑하진',
 '정도가',
 '김미숙',
 '스키장에서',
 '해결되냐',
 '제법이구나',
 '땅에',
 '부자면서',
 '아셔',
 '대주는',
 '애정표현',
 '환',
 '계솔이',
 '본영을',
 '궈서',
 '존대를',
 '박혁권이',
 '내밀어야',
 '봉수가',
 '많음',
 '미현언니한테',
 '부탁해라',
 '싸인회',
 '엄격하게',
 '국화빵',
 '좋아했냐',
 '위험하다고',
 '명왕성',
 '회수',
 '쓸일이',
 '안끝났어',
 '멈춰서고',
 '녀석들도',
 '기막히다',
 '다섯배',
 '헤아리시어',
 '어머니에',
 '김종서가',
 '잡아내',
 '생겼수',
 '난리냐구',
 '미디어',
 '편집장님이랑',
 '열악한',
 '위협하여',
 '안되겠냐구',
 '말밖에',
 '일한',
 '정류장에서',
 '휴일이',
 '좋게도',
 '고백이나',
 '선배야',
 '마라야',
 '잘자',
 '맛인가',
 '헐벗고',
 '울주',
 '불쌍해',
 '상상의',
 '대학교수',
 '지사

In [0]:
장애인도 신체 검사 받나요